#  Jogo BreakThrough

## Introdução à Inteligência Artificial edição 2022/23
### Projeto para avaliação

<img src=".\images\Picture0.png" alt="Drawing" style="width: 200px;"/>

### Grupo: 40

#### Elementos do Grupo

Nome: Francisco Correia

Número: 54685

Nome: Francisco Maia

Número: 55855

Nome: Alexandre Fonseca

Número: 55955

## Formulação do Jogo BreakThrough em termos de estados e de operadores

### Descrição da representação dos estados do jogo

#### A classe de estado: `EstadoBT_40`

In [26]:
class EstadoBT_40:
    def __init__(self, to_move, utility, board, whites, blacks, moves=None):
        self.to_move = to_move
        self.utility = utility
        self.board = board
        self.whites = whites
        self.blacks = blacks
        self.moves = moves

    def __str__(self):
        player_chars = [".", "W", "B"]  # 0, WHITES and BLACKS
        board_str = ["-----------------"]
        for i in range(len(self.board), 0, -1):
            board_str.append(
                f"{i}|" + " ".join(map(lambda x: player_chars[x], self.board[i - 1]))
            )
        board_str.append("-+---------------")
        board_str.append(" |a b c d e f g h")
        return "\n".join(board_str)

<style>
    li {line-height: 1.5em};
</style>
Seja `jogo` uma instância de `JogoBT_40()`. Os estados de `jogo` são representados pela classe `EstadoBT_40`, com os seguintes atributos:
1. `to_move`: o jogador que vai fazer a jogada seguinte (1: brancas, 2: pretas)
2. `utility`: a utilidade neste estado para o jogador anterior.  
  O valor é calculado por [`jogo.compute_utility(row, player)`](./IIA2223-proj-jog-40.py#L116).  
  Um estado do qual resulta a vitória do jogador 1 (brancas) tem o valor `1` (`-1` para as peças pretas, `0` para "em curso").  
  Na prática usa-se este valor previamente calculado para devolver a utilidade via [`jogo.utility(state, player)`](./IIA2223-proj-jog-40.py#L112).
1. `board`: o tabuleiro do jogo, representado por uma matriz quadrada de dimensão arbitrária (`n=8` por omissão).  
  As posições no tabuleiro contêm os identificadores dos jogadores (1: brancas, 2: pretas, 0 posição livre).
1. `whites`: o conjunto das posições ocupadas pelas peças brancas (as do jogador 1).
2. `blacks`: o conjunto das posições ocupadas pelas peças pretas (as do jogador 2).
3. `moves`: a lista de ações válidas para este estado (*i.e.*, para o jogador `to_move`).  
  Este atributo toma o valor `None` por omissão e serve de *cache* para [`jogo.actions(state)`](./IIA2223-proj-jog-40.py#L50).  
  Isto é útil caso queiramos reutilizar as ações (*e.g.*, numa função de avaliação, para *memoization* de estados).  
  O atributo começa a `None` para poupar recursos: no caso do algoritmo *alfabeta*, quando o teste de corte é verdadeiro não se calculam as ações para o estado a avaliar.

De notar que optámos por não usar `GameState` (em [jogos.py](./jogos.py#20)). Como `GameState` é uma sub-classe de um tuplo, não permite herança.  
Por conseguinte, não poderíamos ter atributos mutáveis e um construtor com argumentos por omissão (*i.e.*, `moves=None`).

#### A classe de jogo: `JogoBT_40`

A classe `JogoBT_40` expande a classe `Game` (em [jogos.py](./jogos.py)).  
A funcionalidade da classe é explicada nas secções mais à frente. Por agora podemos só correr a célula de código seguinte.

In [27]:
from jogos import Game

class JogoBT_40(Game):
    WHITE = 1
    BLACK = 2

    def __init__(self, n=8):
        self.n = n
        whites = set((row, col) for row in range(2) for col in range(n))
        blacks = set((row, col) for row in range(n - 2, n) for col in range(n))
        board = [[0 for _ in range(n)] for _ in range(n)]
        for x, y in whites:
            board[x][y] = JogoBT_40.WHITE
        for x, y in blacks:
            board[x][y] = JogoBT_40.BLACK
        to_move = JogoBT_40.WHITE
        self.action_dict = self.compute_action_dict(n)
        self.initial = EstadoBT_40(to_move, 0, board, whites, blacks)

    def actions(self, state: EstadoBT_40):
        if state.moves:
            return state.moves
        moves = set()
        if state.to_move == JogoBT_40.WHITE:
            pieces, pieces_opponent = state.whites, state.blacks
        else:
            pieces, pieces_opponent = state.blacks, state.whites
        for pos in pieces:
            for target, move in self.action_dict[pos][state.to_move].items():
                # can't eat opponent's piece if it is directly in front of ours
                if target[1] == pos[1] and target in pieces_opponent:
                    continue
                if target not in pieces:  # don't eat our own pieces
                    moves.add(move)
        state.moves = sorted(moves)
        return state.moves

    def display(self, state):
        print(state)
        if not self.terminal_test(state):
            print(f'--NEXT PLAYER: {"W" if state.to_move == JogoBT_40.WHITE else "B"}')

    def executa(self, state: EstadoBT_40, valid_actions: "list[str]"):
        """Executa várias jogadas sobre um estado dado.
        Devolve o estado final."""
        result = state
        for move in valid_actions:
            result = self.result(result, move)
        return result

    def result(self, state: EstadoBT_40, move):
        board = [row[:] for row in state.board]  # deepcopy
        (old_row, old_col), (new_row, new_col) = self.convert_move(move)
        board[new_row][new_col] = state.to_move
        board[old_row][old_col] = 0
        whites, blacks = state.whites.copy(), state.blacks.copy()

        if state.to_move == JogoBT_40.WHITE:
            whites.remove((old_row, old_col))
            whites.add((new_row, new_col))
            blacks.discard((new_row, new_col))
            to_move = JogoBT_40.BLACK
        else:
            blacks.remove((old_row, old_col))
            blacks.add((new_row, new_col))
            whites.discard((new_row, new_col))
            to_move = JogoBT_40.WHITE

        return EstadoBT_40(
            to_move,
            self.compute_utility(new_row, state.to_move),
            board,
            whites,
            blacks,
        )

    def terminal_test(self, state: EstadoBT_40):
        # Os próprios jogos em jogar.py lidam com
        # os casos em que já não há jogadas (peças).
        return state.utility != 0

    def utility(self, state: EstadoBT_40, player):
        # W: 1, B: -1
        return state.utility if player == JogoBT_40.WHITE else -state.utility

    def compute_utility(self, row, player):
        """
        Devolve a utilidade para um dado estado:
        (1: vitória, -1: derrota, 0: não terminou)

        Pressupõe-se que a função é chamada unicamente quando
        é criado um novo estado e que o valor de `row` resulta
        de uma jogada válida para o `player` em questão.
        """
        if 0 < row < self.n - 1:
            return 0
        return 1 if player == JogoBT_40.WHITE else -1

    def compute_action_dict(self, n):
        """Devolve um dicionário que associa uma posição `(x, y)`
        usada no estado interno às ações disponíveis nessa posição.

        As associações são feitas em função do jogador
        (`1` para peças brancas, `2` para peças pretas).

        As ações são devolvidas num dicionário em que as chaves são
        a posição do destino e os valores são a ação em si.

        Exemplo de ações para uma peça em `(1,1)` (visualmente, `b2`):
        ```python
        jogo = JogoBT_40()
        actions_b2 = jogo.action_dict[(1, 1)]
        jogo.action_dict[(1, 1)] == (
            None,  # padding para indexar com state.to_move sem offset
            {(2, 0): "b2-a3", (2, 1): "b2-b3", (2, 2): "b2-c3"}, # jog. 1
            {(0, 0): "b2-a1", (0, 1): "b2-b1", (0, 2): "b2-c1"}, # jog. 2
        )
        ```"""

        def in_board(x):
            return 0 <= x < n

        ret = {}
        a_ord = ord("a")
        for row in range(n):
            for col in range(n):
                black_moves = {}
                white_moves = {}
                for i in filter(in_board, (row - 1, row + 1)):
                    # white pieces move upwards, black pieces downwards
                    moves = white_moves if i > row else black_moves
                    for j in filter(in_board, (col - 1, col, col + 1)):
                        moves[(i, j)] = "-".join(
                            [f"{chr(col + a_ord)}{row + 1}", f"{chr(j + a_ord)}{i + 1}"]
                        )
                # use None to pad tuple so we can use to_move as index
                ret[(row, col)] = (None, white_moves, black_moves)
        return ret

    def convert_move(self, move):
        """Converte uma ação no formato do enunciado
        para uma ação representada no estado interno.
        Exemplo: "a1-b2" -> ((0, 0), (1, 1))"""
        ord_a = ord("a")
        (old_col, old_row), (new_col, new_row) = move.split("-")
        return (
            (int(old_row) - 1, ord(old_col) - ord_a),
            (int(new_row) - 1, ord(new_col) - ord_a),
        )

### Testes da formulação

#### Situações iniciais dos jogos

Pretendemos construir um jogo de Breakthrough. A situação inicial de um jogo de Breakthrough é a seguinte:
<div>
<img src=".\images\Picture1.png" alt="Drawing" style="width: 150px;"/><br>
<p style="text-align: center;">Figura 1</p>
</div>

A construção de um novo jogo tem como objetivo definir quais as posições iniciais das peças brancas e das peças pretas.  
As brancas vão preencher as duas linhas de baixo (primeiras duas), e as pretas as duas de cima (últimas duas). 

Antes de mais, definem-se duas constantes na classe para os jogadores, acedidas através de `JogoBT_40.WHITE` e `JogoBT_40.BLACK`:
```python
class JogoBT_40(Game):
    WHITE = 1
    BLACK = 2
```

Geram-se e guardam-se os conjuntos de peças brancas e pretas para o estado inicial:
```python
whites = set((row, col) for row in range(2) for col in range(n))
blacks = set((row, col) for row in range(n - 2, n) for col in range(n))
```

De seguida é criado e preenchido o tabuleiro:
```python
board = [[0 for _ in range(n)] for _ in range(n)]
for x, y in whites:
    board[x][y] = JogoBT_40.WHITE
for x, y in blacks:
    board[x][y] = JogoBT_40.BLACK
```


O primeiro jogador a jogar é o que tem as peças brancas (jogador 1):
```python
to_move = JogoBT_40.WHITE
```

É ainda criado e guardado **o atributo** `action_dict`, um dicionário de ações para um jogo com a dimensão dada:
```python
self.action_dict = self.compute_action_dict(n)

```
Iremos ver a utilidade deste dicionário numa secção mais adiante.

O estado inicial é então construído:
```python
self.initial = EstadoBT_40(to_move, 0, board, whites, blacks)
```

Vamos agora iniciar um jogo e observar o seu estado inicial:

In [28]:
j1 = JogoBT_40()
print("Board:")
for row in j1.initial.board:
    print("  ", row)
print("Próximo jogador:", j1.initial.to_move)
print("Peças brancas:", j1.initial.whites)
print("Peças pretas:", j1.initial.blacks)
print("Utilidade guardada neste estado:", j1.initial.utility)
print("Ações guardadas neste estado:", j1.initial.moves)

Board:
   [1, 1, 1, 1, 1, 1, 1, 1]
   [1, 1, 1, 1, 1, 1, 1, 1]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [2, 2, 2, 2, 2, 2, 2, 2]
   [2, 2, 2, 2, 2, 2, 2, 2]
Próximo jogador: 1
Peças brancas: {(0, 1), (0, 7), (1, 2), (0, 4), (0, 0), (1, 5), (1, 1), (0, 3), (1, 4), (0, 6), (0, 2), (1, 7), (0, 5), (1, 0), (1, 6), (1, 3)}
Peças pretas: {(7, 4), (6, 2), (7, 1), (7, 7), (6, 5), (6, 1), (7, 0), (6, 4), (7, 3), (6, 7), (7, 6), (7, 2), (6, 0), (6, 6), (7, 5), (6, 3)}
Utilidade guardada neste estado: 0
Ações guardadas neste estado: None


### Método Display()

De notar que a representação interna do tabuleiro espelha a representação visual na vertical: onde estão as brancas (1) deviam estar as pretas (2), e vice-versa.  
Isto é simplesmente porque estamos a imprimir por ordem crescente, e os índices internos correspondem aos índices visuais + 1.

Uma representação interna mais semelhante à obtidade pelo método `jogo.display(state)` seria a seguinte:

In [29]:
print("Board com representação interna invertida na vertical:")
for i in range(len(j1.initial.board), 0, -1):
    print("  ", j1.initial.board[i - 1])

Board com representação interna invertida na vertical:
   [2, 2, 2, 2, 2, 2, 2, 2]
   [2, 2, 2, 2, 2, 2, 2, 2]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [0, 0, 0, 0, 0, 0, 0, 0]
   [1, 1, 1, 1, 1, 1, 1, 1]
   [1, 1, 1, 1, 1, 1, 1, 1]


Observemos agora o display do estado inicial do jogo:

In [30]:
j1.display(j1.initial)

-----------------
8|B B B B B B B B
7|B B B B B B B B
6|. . . . . . . .
5|. . . . . . . .
4|. . . . . . . .
3|. . . . . . . .
2|W W W W W W W W
1|W W W W W W W W
-+---------------
 |a b c d e f g h
--NEXT PLAYER: W


### Método actions()

#### Descrição

O método `actions` devolve  ações possiveís e válidas para o próximo jogador. Neste caso, o formato é uma lista do estilo `["a1-b2", "c2-c3", ...]`.

No início do método verificamos se o estado `state` já tem a lista de ações calculada. Neste caso devolvemos logo as ações, e em caso contrário calculamos (e guardamos) as mesmas:
```python
def actions(self, state: EstadoBT_40):
    if state.moves:
        return state.moves
````

A abordagem neste método é iterar sobre todas as peças do jogador a jogar e servir-nos do dicionário de ações gerado no construtor.  
Usamos um conjunto `moves` para evitar ações repetidas; no fim produzimos uma lista ordenada a partir do conjunto.
```python
moves = set()
if state.to_move == JogoBT_40.WHITE:
    pieces, pieces_opponent = state.whites, state.blacks
else:
    pieces, pieces_opponent = state.blacks, state.whites
for pos in pieces:
    for target, move in self.action_dict[pos][state.to_move].items():
```

Ao iterar pelas peças verificamos:
- Se existe uma peça do adversário na posição "à frente" ou "atrás" da peça atual.
- Se **não** existe uma peça amigável na posição para as quais a peça atual se pode mover.

A jogada só é válida se **ambas** as condições forem falsas:

```python
# can't eat opponent's piece if it is directly in front of ours
if target[1] == pos[1] and target in pieces_opponent:
    continue
if target not in pieces:  # don't eat our own pieces
    moves.add(move)
```

No fim guardamos a lista ordenada de ações no estado, para eventualmente reutilizar no futuro, e devolvemos a mesma:
```python
state.moves = sorted(moves)
return state.moves
```

#### Dicionário de ações

A função `compute_action_dict(self, n)` devolve um dicionário que associa uma posição `(x, y)` usada no estado interno às ações disponíveis nessa posição.  
As associações são feitas em função do jogador: devolve-se um tuplo com `(None, <ações peças brancas>,  <ações peças pretas>)`.   
As ações para cada jogador são devolvidas num dicionário em que as <span style="color: plum">chaves</span> são a <span style="color: plum">posição do destino</span> e os <span style="color: MediumSeaGreen">valores</span> são a <span style="color: MediumSeaGreen">ação</span> no formato "visual".

Exemplo de ações para uma peça em `(1,1)` (visualmente, `b2`):

In [31]:
print("Ações para uma peça em '(1,1)' (visualmente, 'b2'):")
padding, acoes_brancas, acoes_pretas = j1.action_dict[(1,1)]
print("Vazio:  ", padding)
print("Brancas:", acoes_brancas)
print("Pretas: ", acoes_pretas)

Ações para uma peça em '(1,1)' (visualmente, 'b2'):
Vazio:   None
Brancas: {(2, 0): 'b2-a3', (2, 1): 'b2-b3', (2, 2): 'b2-c3'}
Pretas:  {(0, 0): 'b2-a1', (0, 1): 'b2-b1', (0, 2): 'b2-c1'}


Vejamos o funcionamento do método `actions`:

In [32]:
to_move = j1.initial.to_move
player = 'W' if to_move == 1 else 'B'
print("Next Player:", player)
print("Ações possíveis:", j1.actions(j1.initial))

Next Player: W
Ações possíveis: ['a2-a3', 'a2-b3', 'b2-a3', 'b2-b3', 'b2-c3', 'c2-b3', 'c2-c3', 'c2-d3', 'd2-c3', 'd2-d3', 'd2-e3', 'e2-d3', 'e2-e3', 'e2-f3', 'f2-e3', 'f2-f3', 'f2-g3', 'g2-f3', 'g2-g3', 'g2-h3', 'h2-g3', 'h2-h3']


## Jogos entre jogadores simples
Nesta secção irão realizar alguns jogos, para verificar a modelização

Jogo entre dois jogadores "random", que escolhem uma ação aleatoriamente:

In [33]:
from jogos import *
from jogar import *

j2 = JogoBT_40()
random_1 = Jogador("Randall", random_player)
random_2 = Jogador("Randy", random_player)
game_to_display = joga11(j2, random_1, random_2)
mostraJogo(j2, game_to_display, False)

Randall vs Randy
Ganham as Whites


Jogo entre um jogador "random" e um jogador "alfabeta", com depth 2, que utiliza como `eval_fun` a `função utility`:

In [34]:
alfabeta_simples_1 = JogadorAlfaBeta("Alfaiate", 2, j2.utility)
mostraJogo(j2, joga11(j2, alfabeta_simples_1, random_1))

Alfaiate vs Randall
Ganham as Blacks


Campeonato entre 4 jogadores, dois "random" e dois "alfabeta", com depth 2, que utilizam a função `utility` como `eval_fun`:

In [35]:
alfabeta_simples_2 = JogadorAlfaBeta("Alberta", 2, j2.utility)
faz_campeonato(j2, [random_1, random_2, alfabeta_simples_1, alfabeta_simples_2], 10)

1 Randall Randy --vencedor= Randall
2 Randall Alfaiate --vencedor= Alfaiate
3 Randall Alberta --vencedor= Alberta
4 Randy Randall --vencedor= Randall
5 Randy Alfaiate --vencedor= Alfaiate
6 Randy Alberta --vencedor= Alberta
7 Alfaiate Randall --vencedor= Alfaiate
8 Alfaiate Randy --vencedor= Alfaiate
9 Alfaiate Alberta --vencedor= Alfaiate
10 Alberta Randall --vencedor= Alberta
11 Alberta Randy --vencedor= Randy
12 Alberta Alfaiate --vencedor= Alberta
JOGADOR VITÓRIAS
Alfaiate       5
Alberta        4
Randall        2
Randy          1


Vejamos o jogo entre os dois jogadores "random" passo-a-passo:

In [36]:
jogadores, jogadas, vencedor = game_to_display
print("Jogo entre", jogadores[0], "e", jogadores[1], "\n")
print("As jogadas efetuadas foram: ", jogadas, "\n")
print("Vencedor do jogo:", jogadores[vencedor - 1])
estado_to_display = j2.initial
for move in jogadas:
    j2.display(estado_to_display)
    estado_to_display = j2.result(estado_to_display, move)

Jogo entre Randall e Randy 

As jogadas efetuadas foram:  ['d2-d3', 'd7-d6', 'd3-d4', 'h7-g6', 'd4-c5', 'e7-f6', 'd1-d2', 'c8-d7', 'c2-c3', 'g6-h5', 'a2-a3', 'g8-h7', 'a1-a2', 'd6-e5', 'a2-b3', 'd8-e7', 'c1-c2', 'h5-g4', 'h2-g3', 'c7-d6', 'g2-h3', 'g7-h6', 'f2-e3', 'f8-g7', 'b1-a2', 'f7-e6', 'c5-c6', 'h6-g5', 'e3-f4', 'e5-e4', 'b3-a4', 'f6-f5', 'c2-d3', 'b7-c6', 'e2-e3', 'b8-b7', 'f4-e5', 'd6-e5', 'a4-b5', 'h7-g6', 'c3-c4', 'g7-h6', 'd3-d4', 'e7-f6', 'a3-a4', 'e5-d4', 'b5-c6', 'b7-b6', 'b2-a3', 'a8-b7', 'g1-f2', 'g4-f3', 'g3-h4', 'h8-h7', 'h1-g2', 'a7-a6', 'c6-d7', 'h6-h5', 'c4-c5', 'f6-e5', 'h4-g5', 'e4-d3', 'c5-b6', 'f5-e4', 'd7-e8'] 

Vencedor do jogo: Randall
-----------------
8|B B B B B B B B
7|B B B B B B B B
6|. . . . . . . .
5|. . . . . . . .
4|. . . . . . . .
3|. . . . . . . .
2|W W W W W W W W
1|W W W W W W W W
-+---------------
 |a b c d e f g h
--NEXT PLAYER: W
-----------------
8|B B B B B B B B
7|B B B B B B B B
6|. . . . . . . .
5|. . . . . . . .
4|. . . . . . . .
3|. .

## Exemplos de jogadores alfabeta
 Descreva e teste nesta secção as várias funções de avaliação desenvolvidas tanto para o ataque como para a defesa.

### Belarmino

Primeiramente, antes de definirmos as nossas próprias funções de avaliação, **definimos o Belarmino**:

In [37]:
def f_aval_belarmino(estado: EstadoBT_40, jogador):
    res = 0
    n = len(estado.board)
    if jogador == JogoBT_40.WHITE:
        for row, _ in estado.whites:
            x = row + 1
            res += x**x
    else:
        for row, _ in estado.blacks:
            x = n - row
            res += x**x
    return res

### Jogadores e funções de avaliação desenvolvidas

#### Jogador *Marco*

O primeiro jogador desenvolvido, vamos chamar-lhe **Marco**, avalia 5 critérios diferentes, estes 5 sendo definidos por funções próprias:
- `(+)` **Vitória**: Se o jogador já venceu.
- `(+)` **Vitória iminente**: Se é possível ganhar numa jogada, ou seja, se temos uma peça na penúltima linha, e esta peça não pode ser "comida" - `função threat()`.
- `(+)` **HomeGround**: Se a peça está na primeira linha;para peças brancas é a linha 1, para peças pretas é a linha 8 (ou *n*).
- `(-)` **Colunas vazias**: Se existem colunas vazias, isto é, que não contenham nenhuma peça do jogador.
- `(-)` **Número de peças do adversário**: O número de peças do adversário.

**Avaliação da vitória**

Esta função serve somente para verificar se existe alguma peça do jogador na linha objetivo, ou seja, se o jogador ganhou. 
Se for verdadeiro retornamos +infinito, pois este é o melhor resultado que podemos obter.

In [38]:
def func_aval_win(estado: EstadoBT_40, jogador):
    """Função de avaliação que devolve um valor arbitrariamente
    elevado se o estado for vitorioso."""
    n = len(estado.board)
    pieces = estado.pieces[jogador - 1]
    target_row = (n - 1, 0)[jogador - 1]
    for row, _ in pieces:
        if target_row == row:
            return float("+inf")
    return 0

**Vitória Eminente**

Esta função verifica se o jogador tem alguma peça na linha antes da sua linha objetivo e esta peça não pode ser "comida" por peças adversárias. Se isto se comprovar o jogador está a uma jogada da vitória, pois na próxima basta avançar essa peça para ganhar.

In [39]:
def func_aval_one_move_to_win(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza uma vitória iminente
    (i.e., a uma jogada de vencer)."""
    res = 0
    n = len(estado.board)
    target, k = ((n - 1, -1), (0, 1))[jogador - 1]
    pieces = estado.pieces[jogador - 1]
    for row, col in pieces:
        if row == target + k and not estado.threat((row, col)):
            res += 1
    return res

**HomeGround**

Neste critério vamos valorizar as peças que estão na primeira linha do jogador, este critério vai ser bastante valorizado, pois é bastante eficiente frente a adversários muito atacante, como o **Belarmino**.

In [40]:
def func_aval_home_ground(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza peças na primeira linha,
    e, por conseguinte, a defesa."""
    n = len(estado.board)
    home = 0 if jogador == JogoBT_40.WHITE else n - 1
    res = 0
    pieces = estado.pieces[jogador - 1]
    for row, _ in pieces:
        if row == home:
            res += 1
    return res

 **Colunas vazias**

 A seguinte função verifica se existem colunas que não têm peças do jogador, é importante verificar este critério de modo a não ter demasiadas peças numa só coluna.

In [41]:
def func_aval_empty_cols(estado: EstadoBT_40, jogador):
    """Função de avaliação que (des)valoriza a existência de
    colunas vazias (isto é, não ocupadas por peças amigáveis)."""
    occupied_cols = set()
    pieces = estado.pieces[jogador - 1]
    n = len(estado.board)
    for _, col in pieces:
        occupied_cols.add(col)
    return n - len(occupied_cols)

**Número de peças do adversário**

Esta função retorna somente o número de peças do adversário.

In [42]:
def func_aval_opponent_piece_count(estado: EstadoBT_40, jogador):
    """Função de avaliação que (des)valoriza o número de
    peças do adversário."""
    pieces_opponent = estado.pieces[jogador % 2]
    return len(pieces_opponent)

Temos por fim o jogador **Marco** que utiliza as funções apresentadas acima.

Este jogador vai ter a sua vertente defensiva ao valorizar bastante as peças na linha mais recuada, enquanto que a sua vertente atacante advém da desvalorização pelo número de peças que o adversário tem, incentivando o algoritmo a atacar mais as peças adversárias.

In [43]:
def func_aval_marco(estado: EstadoBT_40, jogador):
    """Função de avaliação formada a partir das
    diversas funções de avaliação criadas."""
    res = func_aval_win(estado, jogador)
    if res > 0:
        return res

    res -= 20 * func_aval_empty_cols(estado, jogador)

    res -= 80 * func_aval_opponent_piece_count(estado, jogador)

    res += 10000 * func_aval_one_move_to_win(estado, jogador)

    res += 150 * func_aval_home_ground(estado, jogador)

    return res

#### Jogador *Heurácio*

O último jogador desenvolvido é o **Heurácio**. Baseia-se nos jogadores anteriores mas incorpora 6 novos critério.

Sendo estes critérios os seguintes:
- `(+)` **ligações horizontais**: existência de peças amigáveis adjacentes à peça atual na horizontal
- `(+)` **ligações verticais**: existência de peças amigáveis adjacentes à peça atual na vertical
- `(+)` **proteção**: existência de peças amigáveis que podem contra-atacar caso a nossa peça seja comida pelo adversário
- `(-)` **vulnerabilidade**: possibilidade da nossa peça ser comida pelo adversário
- `(+)` **distância até à vitória**: a peça estar na penúltima ou na antepenúltima fila, desde que **não** se encontre **vulnerável**
- `(+)` **valor intrínseco**: atribuem-se pontos a todas as peças (isto é, quantas mais, melhor)

**Ligações horizontais** e **Ligações verticais**

Estes dois critérios são bastante semelhante, no fundo ao juntar as duas, vamos valorizar as peças adjacentes a cada peça. Separamos as horizontais das verticais, pois as horizontais são mais valorizadas que as verticais.

In [44]:
def func_aval_horizontal(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza a existência de
    peças amigáveis adjacentes a uma peça na horizontal."""
    res = 0
    pieces = estado.pieces[jogador - 1]
    for row, col in pieces:
        if (row, col - 1) in pieces or (row, col + 1) in pieces:
            res += 1
    return res


def func_aval_vertical(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza a existência de
    peças amigáveis adjacentes a uma peça na vertical."""
    res = 0
    pieces = estado.pieces[jogador - 1]
    for row, col in pieces:
        if (row - 1, col) in pieces or (row + 1, col) in pieces:
            res += 1
    return res

**Proteção**

A função verifica se existem peças amigáveis a proteger as peças do jogador, ou seja, que podem contra atacar se esta for atacada

In [ ]:
def func_aval_protected(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza a existência de peças amigáveis
    que podem contra-atacar caso uma peça seja comida pelo adversário."""
    res = 0
    pieces = estado.pieces[jogador - 1]
    k = (-1, 1)[jogador - 1]
    for row, col in pieces:
        friend_row = row + k
        if (friend_row, col - 1) in pieces or (friend_row, col + 1) in pieces:
            res += 1
    return res

**Vulnerabilidade** e **distância até à vitória**

Este critério vai ter em conta a possibilidade de uma peça poder ser atacada, desvalorizando caso isto se confirme. Vai também valorizar peças que mesmo podendo ser atacadas, que estejam em posições mais avançadas

In [ ]:
def func_aval_danger(estado: EstadoBT_40, jogador):
    """Função de avaliação que (des)valoriza a possibilidade
    de uma peça ser comida pelo adversário.
    A função valoriza ainda peças que estejam relativamente
    perto da vitória, desde que estas não estejam em perigo."""
    res = 0
    n = len(estado.board)
    k, second_row, third_row = ((1, n - 2, n - 3), (-1, 1, 2))[jogador - 1]
    pieces_opponent = estado.pieces[jogador % 2]
    for row, col in pieces_opponent:
        row = row + k
        if (row, col - 1) in pieces_opponent or (row, col + 1) in pieces_opponent:
            # Peças mais avançadas e que não podem ser atacadas valem mais
            res -= 3
            if row == third_row:  # vs. a 1a fila do adversário
                res += 1
            elif row == second_row:  # vs. a 1a fila do adversário
                res += 4
    return res

**Distância até à vitória**

Função que vai valorizar a posição (linha), das peças do jogador

In [ ]:

def func_aval_piece_value(estado: EstadoBT_40, jogador):
    """Função de avaliação que valoriza o número de
    peças amigáveis."""
    res = 0
    n = len(estado.board)
    pieces = estado.pieces[jogador - 1]
    if jogador == JogoBT_40.WHITE:
        for row, _ in pieces:
            res += row + 1
    else:
        for row, _ in pieces:
            res += n - row
    return res

In [45]:
def func_aval_heuracio(estado: EstadoBT_40, jogador):
    """Função de avaliação formada a partir das
    diversas funções de avaliação criadas."""
    res = func_aval_win(estado, jogador)
    if res > 0:
        return res

    res += 15 * func_aval_horizontal(estado, jogador)

    res += 5 * func_aval_vertical(estado, jogador)

    res += 15 * func_aval_protected(estado, jogador)

    res += 5 * func_aval_danger(estado, jogador)

    res -= 20 * func_aval_empty_cols(estado, jogador)

    res -= 80 * func_aval_opponent_piece_count(estado, jogador)

    res += 10000 * func_aval_one_move_to_win(estado, jogador)

    res += 150 * func_aval_home_ground(estado, jogador)

    res += 10 * func_aval_piece_value(estado, jogador)

    return res

## Exemplos de jogos entre alguns desses jogadores e o Belarmino

### Realização de campeonato com depth 1

Dentro deste campeonato vão existir jogos entre os nosso jogadores também, mas o que pretendemos demonstrar é os resultados dos jogadores frente ao belarmino

In [46]:
depth = 1
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, func_aval_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, func_aval_Heuracio)
faz_campeonato(jogo, [belarmino, marco, heuracio], 10)

NameError: name 'f_aval_jogador_Marco' is not defined

### Realização de campeonato com depth 2
Dentro deste campeonato vão existir jogos entre os nosso jogadores, mas o que pretendemos demonstrar é os resultados dos jogadores frente ao Belarmino.

In [ ]:
depth = 2
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [belarmino, marco, heuracio], 10)

### Realização de jogos com depth 3

*Nota: Não é realizado um campeonato pois este é demorado*

In [ ]:
depth = 3
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)

mostraJogo(jogo, joga11com_timeout(jogo, belarmino, marco, 10), False)
mostraJogo(jogo, joga11com_timeout(jogo, heuracio, belarmino, 10), False)

## Exemplos de jogos entre dois dos vários jogadores desenvolvidos

### Campeonato com depth 1

In [ ]:
depth = 1
jogo = JogoBT_40()
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

### Campeonato com depth 2

In [ ]:
depth = 2
jogo = JogoBT_40()
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

### Campeonato com depth 3

In [ ]:
depth = 3
jogo = JogoBT_40()
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

*Nota: Ao testar, verificamos que é possível realizar jogos com depth 4, mas não vamos incluir pois são muito demorados*

## Processo de selecção dos jogadores para o torneio
Descreva o processo de selecção dos jogadores campeões, para entrar no campeonato "todos contra todos".



A nossa escolha de jogador advém do nosso método de desenvolvimento. Decidimos desenvolver o nosso jogador de forma iterativa, isto é, a partir do primeiro jogador desenvolvido, fomos melhorando a sua função e o peso de cada avaliação.d
Este método de desenvolvimento, deve-se aos ótimos resultados obtidos, frente ao Belarmino, utilizando o primeiro jogador desenvolvido, o jogador **Marco**

**COLOCAR INFORMAÇÃO DE VÁRIOS JOGOS**

Após verificarmos que ganhava grande parte dos seus jogos frente ao Belarmino, verificamos que mesmo assim os movimentos do Marco eram demasiado defensivos e pouco eficazes contra um adversário mais inteligente. Com essa observação, desenvolvemos a função ``` piece_value()```, esta permite valorizar a peça, tendo em conta a sua posição.

Como foi explicado acima, a função torna o jogador balanceado no ataque e na defesa. Mesmo assim ao testar-mos contra o Belarmino, o Belarmino vencia bastantes jogos.

**COLOCAR INFORMAÇÃO DE VÁRIOS JOGOS COM HEURÁCIO COM OS VALORES ANTIGOS**

Fizemos mais testes e chegamos à conclusão que os pesos de cada critério teria de ser alterado, fizemos vários testes com valores diferentes, até que chegamos aos valores finais. Realizámos vários campeonatos entre o Belarmino, o Marco, e o Heurácio. Os resultados dos campenatos comprovam que o Heurácio é o melhor, mesmo não ganhando todos os jogos (nem a melhor equipa do mundo ganha todos os jogos), ganha uma percentagem bastante satisfatória

**COLOCAR CAMPEONATOS ENTRE OS 3**

Sendo assim, a nossa escolha recai sobre o Heurácio através de um desenvolvimento iterativo e de vários jogos frente ao belarmino e "versões passadas" do Heurácio

<span style="color:magenta"> É a parte mais importante do relatório, justificando porque convocaram o vosso Ronaldo para o campeonato. Se jogaram com vários jogadores (ou seja, várias funções de avaliação) e fizeram um torneio privado de selecção, podem apresentar aqui uma tabela com esses dados. </span>